# **LIBRARY PREPARATION**

In [ ]:
import pandas as pd
import numpy as np
import torch
import math
import os
import glob
import random

# **LABEL PREPROCESSING**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Saigonmec/original_data/cleaned_original.csv')
df

,Unnamed: 0,image,xmin,ymin,xmax,ymax,label
0,2,10006732_20220608_MLO_R_2.png,750,944,1296,1439,mass
1,3,10006732_20220608_MLO_L_3.png,686,1046,791,1166,microcalcification
2,4,10006732_20220608_CC_R_0.png,828,798,1373,1382,mass
3,5,10006732_20220608_CC_L_1.png,424,921,520,1050,microcalcification
4,6,10006732_20220608_CC_L_1.png,709,999,720,1013,benign
...,...,...,...,...,...,...,...
2835,2874,2211012951_20221208_MLO_L_4.png,98,968,381,1181,microcalcification
2836,2875,2211012951_20221208_MLO_L_4.png,21,137,222,461,lymph_node
2837,2876,2211012951_20221208_CC_L_7.png,105,786,501,1166,mass
2838,2877,2211012951_20221208_CC_L_7.png,122,904,441,1100,microcalcification


In [ ]:
df['calcification'] = df.label.apply(lambda x: 1 if x == "microcalcification" else 0)
df

,Unnamed: 0,image,xmin,ymin,xmax,ymax,label,calcification
0,2,10006732_20220608_MLO_R_2.png,750,944,1296,1439,mass,0
1,3,10006732_20220608_MLO_L_3.png,686,1046,791,1166,microcalcification,1
2,4,10006732_20220608_CC_R_0.png,828,798,1373,1382,mass,0
3,5,10006732_20220608_CC_L_1.png,424,921,520,1050,microcalcification,1
4,6,10006732_20220608_CC_L_1.png,709,999,720,1013,benign,0
...,...,...,...,...,...,...,...,...
2835,2874,2211012951_20221208_MLO_L_4.png,98,968,381,1181,microcalcification,1
2836,2875,2211012951_20221208_MLO_L_4.png,21,137,222,461,lymph_node,0
2837,2876,2211012951_20221208_CC_L_7.png,105,786,501,1166,mass,0
2838,2877,2211012951_20221208_CC_L_7.png,122,904,441,1100,microcalcification,1


In [ ]:
df_final = df.drop(['Unnamed: 0', 'xmin', 'ymin', 'xmax', 'ymax', 'label'], axis = 1)
df_final

,image,calcification
0,10006732_20220608_MLO_R_2.png,0
1,10006732_20220608_MLO_L_3.png,1
2,10006732_20220608_CC_R_0.png,0
3,10006732_20220608_CC_L_1.png,1
4,10006732_20220608_CC_L_1.png,0
...,...,...
2835,2211012951_20221208_MLO_L_4.png,1
2836,2211012951_20221208_MLO_L_4.png,0
2837,2211012951_20221208_CC_L_7.png,0
2838,2211012951_20221208_CC_L_7.png,1


In [ ]:
df_final.to_csv('/content/drive/MyDrive/Saigonmec/original_data/bi-calcification.txt', sep =' ', header=None, index=None)

In [ ]:
import glob
print(len(glob.glob('/content/drive/MyDrive/Saigonmec/original_data/train/*')))

1720


# **CLEAN DUPLICATE**

In [ ]:
sample_dict = {}
sample_dict['mot'] = ['1']
sample_dict['mot'].append('2')
sample_dict

# df_final[]

{'mot': ['1', '2']}

In [ ]:
sample_dict = {}

for i in range(len(df_final)):
  if df_final['image'][i] not in sample_dict:
    sample_dict[df_final['image'][i]]  = list(str(df_final['calcification'][i]))
  else:
    sample_dict[df_final['image'][i]].append(str(df_final['calcification'][i]))

print(len(sample_dict))

1720


In [ ]:
for i in sample_dict:
  if '1' in sample_dict[i]:
    sample_dict[i] = 1
  else:
    sample_dict[i] = 0

In [ ]:
counter_0 = 0
counter_1 = 0
for i in sample_dict:
  if sample_dict[i] == 1:
    counter_1 += 1
  elif sample_dict[i] == 0:
    counter_0 += 1

In [ ]:
print("Number of non-calcification: ",counter_0)
print("Number of calcification: ",counter_1)

Number of non-calcification:  1383
Number of calcification:  337


In [ ]:
sample_dict

{'10006732_20220608_MLO_R_2.png': 0,
 '10006732_20220608_MLO_L_3.png': 1,
 '10006732_20220608_CC_R_0.png': 0,
 '10006732_20220608_CC_L_1.png': 1,
 '10023439_20221201_MLO_R_5.png': 0,
 '10023439_20221201_MLO_L_4.png': 0,
 '10023439_20221201_CC_R_6.png': 0,
 '10023439_20221201_CC_L_7.png': 0,
 '10031935_20220907_MLO_L_0.png': 0,
 '10031935_20220907_CC_L_2.png': 0,
 '10048861_20220907_MLO_L_3.png': 0,
 '10048861_20220907_CC_L_1.png': 0,
 '10049935_20220907_MLO_R_0.png': 0,
 '10049935_20220907_MLO_L_3.png': 0,
 '10049935_20220907_CC_R_2.png': 0,
 '10049935_20220907_CC_L_1.png': 0,
 '11068224_20221206_MLO_L_7.png': 0,
 '11068224_20221206_CC_R_4.png': 0,
 '11068224_20221206_CC_L_5.png': 0,
 '11068224_20221206_MLO_R_6.png': 0,
 '11092788_20220920_CC_R_4.png': 0,
 '11092788_20220920_MLO_R_6.png': 0,
 '11097027_20221208_MLO_R_5.png': 0,
 '11097027_20221208_MLO_L_4.png': 0,
 '11097027_20221208_CC_R_7.png': 0,
 '11097027_20221208_CC_L_6.png': 0,
 '11108914 NG4_20221017_MLO_R_4.png': 0,
 '11108914

In [ ]:
with open('/content/drive/MyDrive/Saigonmec/Calc_Cls/calc_test_2.txt', 'w') as f:
  for i in sample_dict: 
    f.write(i)
    f.write(' '+ str(sample_dict[i]) + '\n')

# **SHUFFLE THEN SPLITTING TRAIN-TEST**

In [ ]:
test_dict = {}
counter_0 = 0
counter_1 = 0
for i in sample_dict:
  if sample_dict[i] == 1:
    test_dict
    counter_1 += 1
  elif sample_dict[i] == 0:
    counter_0 += 1
  if counter_0 = 50 and counter_1 = 50:
    break


In [ ]:
import random

# This shuffling is not in-place
l = list(sample_dict.items())
random.shuffle(l)
d_shuffled = dict(l)
d_shuffled

In [ ]:
test_dict = {}
train_dict = {}
counter_0 = 0
counter_1 = 0
for i in sample_dict:
  if sample_dict[i] == 1 and counter_1 <= 49:
    test_dict[i] = 1
    counter_1 += 1
  if sample_dict[i] == 0 and counter_0 <= 49:
    test_dict[i] = 0
    counter_0 += 1

  if sample_dict[i] == 1 and counter_1 >= 50:
    train_dict[i] = 1
    counter_1 += 1
  if sample_dict[i] == 0 and counter_0 >= 50:
    train_dict[i] = 0
    counter_0 += 1

In [ ]:
print(len(test_dict))
print(len(train_dict))

100
1622


In [ ]:
test_dict

In [ ]:
train_dict

In [ ]:
counter_0 = 0
counter_1 = 0
for i in test_dict:
  if test_dict[i] == 1:
    counter_1 += 1
  elif test_dict[i] == 0:
    counter_0 += 1
print("Number of non-calcification: ",counter_0)
print("Number of calcification: ",counter_1)
counter_0 = 0
counter_1 = 0
for i in train_dict:
  if train_dict[i] == 1:
    counter_1 += 1
  elif train_dict[i] == 0:
    counter_0 += 1
print("Number of non-calcification: ",counter_0)
print("Number of calcification: ",counter_1)

Number of non-calcification:  50
Number of calcification:  50
Number of non-calcification:  1334
Number of calcification:  288


In [ ]:
for i in test_dict:
  for j in train_dict:
    if i == j:
      print(i)

13135322_20200528_CC_R_0.png
2102003005  NG4_20221027_CC_L_5.png


In [ ]:
with open('/content/drive/MyDrive/Saigonmec/Calc_Cls/dataset/calc_train.txt', 'w') as f:
  for i in train_dict: 
    f.write(i)
    f.write(' '+ str(train_dict[i]) + '\n')

In [ ]:
with open('/content/drive/MyDrive/Saigonmec/Calc_Cls/dataset/calc_test.txt', 'w') as f:
  for i in test_dict: 
    f.write(i)
    f.write(' '+ str(test_dict[i]) + '\n')

# **Handling with spaces**

In [ ]:
import glob
import os
def space_clean(old_folder, new_folder):
  old_list_names = glob.glob(old_folder + '/*')
  for i in old_list_names:
    name = i.split('/')[-1]
    raw_list = name.split(" ")
    if len(raw_list) >= 2:
      new_name = raw_list[0] + raw_list[-1]
    else:
      new_name = raw_list[0]
    new_destination = new_folder + '/' + new_name
    print("\nOld: ",i)
    print("New: ",new_destination)
    os.rename(i, new_destination)


space_clean('/content/drive/MyDrive/Saigonmec/original_data/train', '/content/drive/MyDrive/Saigonmec/original_data/train_space_cleaned')

In [ ]:
import glob
print(len(glob.glob('/content/drive/MyDrive/Saigonmec/original_data/train_space_cleaned/*')))

1720


In [ ]:
check_list = glob.glob('/content/drive/MyDrive/Saigonmec/original_data/train_space_cleaned/*')
check_name = []
for i in check_list:
  j = i.split("/")[-1]
  check_name.append(j)
# print(check_name)
for i in check_name:
  if "12097767" in i:
    print(i)

12097767DV_20221007_MLO_R_4.png
12097767DV_20221007_CC_R_5.png
12097767DV_20221007_MLO_L_7.png


# **TRAIN**

In [ ]:
!pip install pytorch_metric_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
!python '/content/drive/MyDrive/Saigonmec/Calc_Cls/train.py' -m resnet50

Adjusting learning rate of group 0 to 1.0000e-03.
Training 1 epoch
100% 102/102 [02:32<00:00,  1.50s/it]
Adjusting learning rate of group 0 to 1.0000e-03.
Evaluating model on test data
100% 7/7 [00:10<00:00,  1.50s/it]
Test set: Avg Test CE Loss: 1.2438; Avg Test Metric Loss: 0.0000; Test accuracy: 50.00%
Saving checkpoint
Saving best model
Training 2 epoch
100% 102/102 [02:37<00:00,  1.55s/it]
Adjusting learning rate of group 0 to 1.0000e-03.
Evaluating model on test data
100% 7/7 [00:10<00:00,  1.48s/it]
Test set: Avg Test CE Loss: 1.8129; Avg Test Metric Loss: 0.0000; Test accuracy: 50.00%
Saving checkpoint
Training 3 epoch
100% 102/102 [02:37<00:00,  1.54s/it]
Adjusting learning rate of group 0 to 1.0000e-03.
Evaluating model on test data
100% 7/7 [00:10<00:00,  1.49s/it]
Test set: Avg Test CE Loss: 1.0735; Avg Test Metric Loss: 0.0000; Test accuracy: 50.00%
Saving checkpoint
Training 4 epoch
100% 102/102 [02:38<00:00,  1.55s/it]
Adjusting learning rate of group 0 to 1.0000e-03.
Ev